In [ ]:
#Utilitarios
from sklearn.model_selection import train_test_split
from mne.externals.pymatreader import read_mat
from sklearn.preprocessing import MaxAbsScaler
from sklearn.decomposition import FastICA
from sklearn import preprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import os

#Rede Neural
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dense, Conv2D
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Dropout, BatchNormalization 
import keras_tuner as kt
from tensorflow import keras
os.environ['TENSORBOARD_BINARY'] = '/path/to/envs/my_env/bin/tensorboard'
%load_ext tensorboard

In [ ]:
subject1_01 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject1_mat\train_subject1_raw01.mat')
subject1_02 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject1_mat\train_subject1_raw02.mat')
subject1_03 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject1_mat\train_subject1_raw03.mat')

subject2_01 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject2_mat\train_subject2_raw01.mat')
subject2_02 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject2_mat\train_subject2_raw02.mat')
subject2_03 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject2_mat\train_subject2_raw03.mat')

subject3_01 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject3_mat\train_subject3_raw01.mat')
subject3_02 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject3_mat\train_subject3_raw02.mat')
subject3_03 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject3_mat\train_subject3_raw03.mat')

In [ ]:
subject3_03.keys()

In [ ]:
#subject3_03['nfo']

In [ ]:
subject_X = np.concatenate( (subject1_01['X'], subject1_02['X'], subject1_03['X'], subject2_01['X'], subject2_02['X'], subject2_03['X'], subject3_01['X'], subject3_02['X'], subject3_03['X'] ), axis = 0)

In [ ]:
subject_Y = np.concatenate( (subject1_01['Y'], subject1_02['Y'], subject1_03['Y'], subject2_01['Y'], subject2_02['Y'], subject2_03['Y'], subject3_01['Y'], subject3_02['Y'], subject3_03['Y'] ), axis = 0)
subject_Y.shape

In [ ]:
#subject_X = MaxAbsScaler().fit_transform( subject_X )

In [ ]:
#mat_data['nfo']['ypos']
#pd.DataFrame( mat_data['X'] ).head()

In [ ]:
# Pre-processamento dos dados
ICA = FastICA()

subject_X = ICA.fit_transform(subject_X)

In [ ]:
Y = np.take( subject_Y , np.arange(0, len( subject_X ), 512) , axis=None) 

In [ ]:
X = np.asarray(  np.split( np.transpose( subject_X ), 2141, axis = 1 ))
X.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42, stratify = Y) 

In [ ]:
NUM_CLASSES = 3

le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
#y_val = le.fit_transform(y_val)
y_test = le.fit_transform(y_test)

#Faz One-Hot Encoding
y_train = to_categorical(y_train, NUM_CLASSES)
#y_val = to_categorical(y_val, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

#Hiperparameters [Concorrente]
alpha = 10^-3 | beta_01 = 0.9 | beta_02 = 0.99 | Camadas = 5 : 512 . 256 . 128 . 64 . 3 | Active = ReLU | Output = Softmax 
Loss Function: categorical cross-entropy | Otimizador: ADAM    

In [ ]:
#Precision For each class 
Precision0 = tf.keras.metrics.Precision(class_id=0)
Precision1 = tf.keras.metrics.Precision(class_id=1)
Precision2 = tf.keras.metrics.Precision(class_id=2)


#Recall For each class
Recall0 = tf.keras.metrics.Recall( class_id=0 )
Recall1 = tf.keras.metrics.Recall( class_id=1 )
Recall2 = tf.keras.metrics.Recall( class_id=2 )

Metricas = ['accuracy', Precision0, Precision1, Precision2, Recall0, Recall1, Recall2]

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add( Flatten(input_shape=(32, 512)) ) #Entrada
    
    model.add( Dense( 512, activation='relu',   name='dense_1') ) #Camada 01
    #model.add( Dropout( hp.Choice('Dropout01', [0.85 ]) ) )
    
    model.add( Dense( 256, activation='relu',   name='dense_2') ) #Camada 02
    #model.add( Dropout( hp.Choice('Dropout02', [ 0.8 ]) ) )
    
    model.add( Dense( 128, activation='relu',   name='dense_2') ) #Camada 02
    #model.add( Dropout( hp.Choice('Dropout03', [0.2]) ) )
    
    model.add( Dense( 64, activation='relu',   name='dense_2') ) #Camada 02
    #model.add( Dropout( hp.Choice('Dropout04', [ 0.1]) ) ) 
    
    model.add( Dense(3, activation='softmax') ) #Saída  
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=Metricas)
    return model

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=9999999,
    overwrite = True,
#    directory="./tf_board/tb_logs" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
)

In [ ]:
tuner.search_space_summary(extended=True)

In [ ]:
# tf.keras.backend.clear_session()
# %store    
# %store -z | relu, sigmoid, softmax, softplus, softsign, tanh, selu, elu, exponential   

In [ ]:
tuner.search(
    x_train, 
    y_train, 
    epochs=100, 
    batch_size = 128,
    shuffle = True,
    validation_split = 0.35,
    #validation_data=(x_val, y_val),
    verbose = 1,
    #callbacks=[keras.callbacks.TensorBoard( log_dir = "./tf_board/tb_logs", histogram_freq=1 )] 
)

In [ ]:
#Colocar estes comandos no prompt

#!cd c:\users\alex\appdata\roaming\python\python39\site-packages\tensorboard
#!python main.py --logdir=C:\Users\Alex\tf_board\tb_logs
#http://localhost:6006/

#[esse não funciona]: %tensorboard --logdir C:\Users\Alex\tf_board\

In [ ]:
tuner.results_summary(num_trials=9999999999)

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
Modelo_Final = tuner.hypermodel.build(best_hp)

In [ ]:
Modelo_Final.fit(x_train
 , y_train
 , batch_size = 128
 , epochs = 100
 , shuffle = True
 )

In [ ]:
#model.summary()

In [ ]:
#Avaliando o modelo
Modelo_Final.evaluate(x_test, y_test)

# Fim